<a href="https://colab.research.google.com/github/LevWilliams/ContradictoryMyDearWatson/blob/master/Contradictory%2C_My_Dear_Watson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Data

In [1]:
!pip install --upgrade kaggle

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


In [2]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [3]:
!kaggle competitions download -c contradictory-my-dear-watson

  0% 0.00/536k [00:00<?, ?B/s]
100% 536k/536k [00:00<00:00, 81.3MB/s]
  0% 0.00/1.23M [00:00<?, ?B/s]
100% 1.23M/1.23M [00:00<00:00, 83.1MB/s]
  0% 0.00/66.0k [00:00<?, ?B/s]
100% 66.0k/66.0k [00:00<00:00, 59.9MB/s]


Modules & Libraries

In [4]:
## importing packages
import gc
import os
import random
!pip install transformers
import transformers
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K

from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel

print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers version: {transformers.__version__}")

warnings.filterwarnings("ignore")

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 1.1MB 51.3MB/s 
     |████████████████████████████████| 890kB 63.3MB/s 
     |████████████████████████████████| 3.0MB 53.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=8c0dbbcf3fc5c8072721b6abd1ccb90ed5ece2cb506b11e7764337e2ea763aed
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
TensorFlow version: 2.3.0
Transformers version: 3.0.2


Configuration

In [5]:
## defining configuration
class Configuration():
    """
    All configuration for running an experiment
    """
    def __init__(
        self,
        model_name,
        max_length = 64,
        padding = True,
        batch_size = 128,
        epochs = 5,
        metrics = ["sparse_categorical_accuracy"],
        verbose = 1,
        train_splits = 5,
        accelerator = "TPU",
        myluckynumber = 13
    ):
        # seed and accelerator
        self.SEED = myluckynumber
        self.ACCELERATOR = accelerator

        # paths
        self.PATH_TRAIN = Path("train.csv.zip")
        self.PATH_TEST  = Path("test.csv.zip")

        # splits
        self.TRAIN_SPLITS = train_splits

        # mapping of language
        self.LANGUAGE_MAP = {
            "English"   : 0,
            "Chinese"   : 1,
            "Arabic"    : 2,
            "French"    : 3,
            "Swahili"   : 4,
            "Urdu"      : 5,
            "Vietnamese": 6,
            "Russian"   : 7,
            "Hindi"     : 8,
            "Greek"     : 9,
            "Thai"      : 10,
            "Spanish"   : 11,
            "German"    : 12,
            "Turkish"   : 13,
            "Bulgarian" : 14
        }

        self.INVERSE_LANGUAGE_MAP = {v: k for k, v in self.LANGUAGE_MAP.items()}

        # model configuration
        self.MODEL_NAME = model_name
        self.TOKENIZER = AutoTokenizer.from_pretrained(self.MODEL_NAME)

        # model hyperparameters
        self.MAX_LENGTH = max_length
        self.PAD_TO_MAX_LENGTH = padding
        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.METRICS = metrics
        self.VERBOSE = verbose
        
        # initializing accelerator
        self.initialize_accelerator()
        
    def initialize_accelerator(self):
        """
        Initializing accelerator
        """
        # checking TPU first
        if self.ACCELERATOR == "TPU":
            print("Connecting to TPU")
            try:
                tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
                print(f"Running on TPU {tpu.master()}")
            except ValueError:
                print("Could not connect to TPU")
                tpu = None

            if tpu:
                try:
                    print("Initializing TPU")
                    tf.config.experimental_connect_to_cluster(tpu)
                    tf.tpu.experimental.initialize_tpu_system(tpu)
                    self.strategy = tf.distribute.experimental.TPUStrategy(tpu)
                    self.tpu = tpu
                    print("TPU initialized")
                except _:
                    print("Failed to initialize TPU")
            else:
                print("Unable to initialize TPU")
                self.ACCELERATOR = "GPU"

        # default for CPU and GPU
        if self.ACCELERATOR != "TPU":
            print("Using default strategy for CPU and single GPU")
            self.strategy = tf.distribute.get_strategy()

        # checking GPUs
        if self.ACCELERATOR == "GPU":
            print(f"GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

        # defining replicas
        self.AUTO = tf.data.experimental.AUTOTUNE
        self.REPLICAS = self.strategy.num_replicas_in_sync
        print(f"REPLICAS: {self.REPLICAS}")

Data Prep

In [6]:
## data preparation functions
def encode_text(df, tokenizer, max_len, padding):
    """
    Preprocessing textual data into encoded tokens
    """
    text = df[["premise", "hypothesis"]].values.tolist()

    # encoding text using tokenizer of the model
    text_encoded = tokenizer.batch_encode_plus(
        text,
        pad_to_max_length = padding,
        max_length = max_len
    )

    return text_encoded


def get_tf_dataset(X, y, auto, labelled = True, repeat = False, shuffle = False, batch_size = 128):
    """
    Creating tf.data.Dataset for TPU
    """
    if labelled:
        ds = (tf.data.Dataset.from_tensor_slices((X["input_ids"], y)))
    else:
        ds = (tf.data.Dataset.from_tensor_slices(X["input_ids"]))

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(2048)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(auto)

    return ds

Model Architecture

In [7]:
## building model
def build_model(model_name, max_len, metrics):
    """
    Building the Deep Learning architecture
    """
    # defining encoded inputs
    input_ids = Input(shape = (max_len,), dtype = tf.int32, name = "input_ids")
    
    # defining transformer model embeddings
    transformer_model = TFAutoModel.from_pretrained(model_name)
    transformer_embeddings = transformer_model(input_ids)[0]
    
    # defining output layer
    output_values = Dense(3, activation = "softmax")(transformer_embeddings[:, 0, :])

    # defining model
    model = Model(inputs = input_ids, outputs = output_values)
    opt = Adam(learning_rate = 1e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    metrics = metrics

    model.compile(optimizer = opt, loss = loss, metrics = metrics)

    return model

Stratified K-Fold Modeling

In [8]:
## stratified k-fold over language and label
def run_model(config):
    """
    Running the model
    """
    ## reading data
    df_train = pd.read_csv(config.PATH_TRAIN)
    df_test = pd.read_csv(config.PATH_TEST)

    # adding column for stratified splitting
    df_train["language_label"] = df_train.language.astype(str) + "_" + df_train.label.astype(str)

    # stratified K-fold on language and label
    skf = StratifiedKFold(n_splits = config.TRAIN_SPLITS, shuffle = True, random_state = config.SEED)

    # initializing predictions
    preds_oof = np.zeros((df_train.shape[0], 3))
    preds_test = np.zeros((df_test.shape[0], 3))
    acc_oof = []

    # iterating over folds
    for (fold, (train_index, valid_index)) in enumerate(skf.split(df_train, df_train.language_label)):
        # initializing TPU
        if config.ACCELERATOR == "TPU":
            if config.tpu:
                config.initialize_accelerator()

        # building model
        K.clear_session()
        with config.strategy.scope():
            model = build_model(config.MODEL_NAME, config.MAX_LENGTH, config.METRICS)
            if fold == 0:
                print(model.summary())

        print("\n")
        print("#" * 19)
        print(f"##### Fold: {fold + 1} #####")
        print("#" * 19)

        # splitting data into training and validation
        X_train = df_train.iloc[train_index]
        X_valid = df_train.iloc[valid_index]

        y_train = X_train.label.values
        y_valid = X_valid.label.values

        print("\nTokenizing")

        # encoding text data using tokenizer
        X_train_encoded = encode_text(df = X_train, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)
        X_valid_encoded = encode_text(df = X_valid, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)

        # creating TF Dataset
        ds_train = get_tf_dataset(X_train_encoded, y_train, config.AUTO, repeat = True, shuffle = True, batch_size = config.BATCH_SIZE * config.REPLICAS)
        ds_valid = get_tf_dataset(X_valid_encoded, y_valid, config.AUTO, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)

        n_train = X_train.shape[0]

        if fold == 0:
            X_test_encoded = encode_text(df = df_test, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)

        # saving model at best accuracy epoch
        sv = tf.keras.callbacks.ModelCheckpoint(
            "model.h5",
            monitor = "val_sparse_categorical_accuracy",
            verbose = 0,
            save_best_only = True,
            save_weights_only = True,
            mode = "max",
            save_freq = "epoch"
        )

        print("\nTraining")

        # training model
        model_history = model.fit(
            ds_train,
            epochs = config.EPOCHS,
            callbacks = [sv],
            steps_per_epoch = n_train / config.BATCH_SIZE // config.REPLICAS,
            validation_data = ds_valid,
            verbose = config.VERBOSE
        )

        print("\nValidating")

        # scoring validation data
        model.load_weights("model.h5")
        ds_valid = get_tf_dataset(X_valid_encoded, -1, config.AUTO, labelled = False, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)

        preds_valid = model.predict(ds_valid, verbose = config.VERBOSE)
        acc = accuracy_score(y_valid, np.argmax(preds_valid, axis = 1))

        preds_oof[valid_index] = preds_valid
        acc_oof.append(acc)

        print("\nInferencing")

        # scoring test data
        ds_test = get_tf_dataset(X_test_encoded, -1, config.AUTO, labelled = False, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)
        preds_test += model.predict(ds_test, verbose = config.VERBOSE) / config.TRAIN_SPLITS

        print(f"\nFold {fold + 1} Accuracy: {round(acc, 4)}\n")

        g = gc.collect()

    # overall CV score and standard deviation
    print(f"\nCV Mean Accuracy: {round(np.mean(acc_oof), 4)}")
    print(f"CV StdDev Accuracy: {round(np.std(acc_oof), 4)}\n")

    return preds_oof, preds_test

Experimenting With Different Models

In [14]:
 #Model 1: Multilingual Bert Base Cased
#config_1 = Configuration("bert-base-multilingual-cased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
#preds_train_1, preds_test_1 = run_model(config_1)

#Model 2: Distilbert Base Uncased
#config_2 = Configuration("distilbert-base-uncased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
#preds_train_2, preds_test_2 = run_model(config_2)

# Model 3: XLM Roberta Base
config_3 = Configuration("distilbert-base-multilingual-cased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
preds_train_3, preds_test_3 = run_model(config_3)


Connecting to TPU
Running on TPU grpc://10.32.112.66:8470
Initializing TPU


INFO:tensorflow:Initializing the TPU system: grpc://10.32.112.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.32.112.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...


Training
Epoch 1/2
 2/23 [=>............................] - ETA: 2:40 - loss: 1.1060 - sparse_categorical_accuracy: 0.3477WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0048s vs `on_train_batch_end` time: 0.0826s). Check your callbacks.


23/23 [==============================] - ETA: 0s - loss: 1.0996 - sparse_categorical_accuracy: 0.3466WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0025s vs `on_test_batch_end` time: 0.0491s). Check your callbacks.


23/23 [==============================] - 33s 1s/step - loss: 1.0996 - sparse_categorical_accuracy: 0.3466 - val_loss: 1.0953 - val_sparse_categorical_accuracy: 0.3553
Epoch 2/2
23/23 [==============================] - 5s 215ms/step - loss: 1.0840 - sparse_categorical_accuracy: 0.3952 - val_loss: 1.0830 - val_sparse_categorical_accuracy: 0.4023

Validating
2/6 [=========>....................] - ETA: 10sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0024s vs `on_predict_batch_end` time: 0.0508s). Check your callbacks.


6/6 [==============================] - 10s 2s/step

Inferencing
2/6 [=========>....................] - ETA: 0sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0025s vs `on_predict_batch_end` time: 0.0494s). Check your callbacks.


6/6 [==============================] - 3s 460ms/step

Fold 2 Accuracy: 0.4023


CV Mean Accuracy: 0.4016
CV StdDev Accuracy: 0.0007



Tuning Hyperparameters

In [ ]:


# Hyperparameter Space 1
#config_1 = Configuration("bert-base-multilingual-cased", max_length = 32, batch_size = 32, epochs = 2, train_splits = 2)
#preds_train_1, preds_test_1 = run_model(config_1)

# Hyperparameter Space 2
config_2 = Configuration("bert-base-multilingual-cased", max_length = 64, batch_size = 32, epochs = 3, train_splits = 2)
preds_train_2, preds_test_2 = run_model(config_2)

# Hyperparameter Space 3
#config_3 = Configuration("bert-base-multilingual-cased", max_length = 84, batch_size = 16, epochs = 4, train_splits = 2)
#preds_train_3, preds_test_3 = run_model(config_3)



Connecting to TPU
Running on TPU grpc://10.38.15.26:8470
Initializing TPU


INFO:tensorflow:Initializing the TPU system: grpc://10.38.15.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.38.15.26:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

 2/23 [=>............................] - ETA: 7:04 - loss: 1.0940 - sparse_categorical_accuracy: 0.3574 WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0107s vs `on_train_batch_end` time: 0.1787s). Check your callbacks.


23/23 [==============================] - ETA: 0s - loss: 1.1042 - sparse_categorical_accuracy: 0.3400WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0043s vs `on_test_batch_end` time: 0.1347s). Check your callbacks.


23/23 [==============================] - 70s 3s/step - loss: 1.1042 - sparse_categorical_accuracy: 0.3400 - val_loss: 1.0991 - val_sparse_categorical_accuracy: 0.3361
Epoch 2/3
23/23 [==============================] - 10s 424ms/step - loss: 1.0946 - sparse_categorical_accuracy: 0.3614 - val_loss: 1.0937 - val_sparse_categorical_accuracy: 0.3703
Epoch 3/3
23/23 [==============================] - 10s 429ms/step - loss: 1.0833 - sparse_categorical_accuracy: 0.4018 - val_loss: 1.0771 - val_sparse_categorical_accuracy: 0.4150

Validating
2/6 [=========>....................] - ETA: 15sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0034s vs `on_predict_batch_end` time: 0.1283s). Check your callbacks.


6/6 [==============================] - 16s 3s/step

Inferencing
2/6 [=========>....................] - ETA: 0sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0035s vs `on_predict_batch_end` time: 0.1264s). Check your callbacks.


6/6 [==============================] - 5s 905ms/step

Fold 2 Accuracy: 0.415


CV Mean Accuracy: 0.4847
CV StdDev Accuracy: 0.0696



Final Model

In [15]:


# Final Model: XLM Roberta Large
#config_1 = Configuration("jplu/tf-xlm-roberta-large", max_length = 84, batch_size = 32, epochs = 16, train_splits = 4)
#preds_train_1, preds_test_1 = run_model(config_1)

config_2 = Configuration("jplu/tf-xlm-r-ner-40-lang", max_length = 84, batch_size = 32, epochs = 32, train_splits = 10)
preds_train_2, preds_test_2 = run_model(config_2)


Connecting to TPU
Running on TPU grpc://10.32.112.66:8470
Initializing TPU


INFO:tensorflow:Initializing the TPU system: grpc://10.32.112.66:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.32.112.66:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

 2/42 [>.............................] - ETA: 12:21 - loss: 1.1119 - sparse_categorical_accuracy: 0.3496WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0086s vs `on_train_batch_end` time: 0.2295s). Check your callbacks.


42/42 [==============================] - ETA: 0s - loss: 1.1056 - sparse_categorical_accuracy: 0.3386WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0038s vs `on_test_batch_end` time: 6.3180s). Check your callbacks.


42/42 [==============================] - 73s 2s/step - loss: 1.1056 - sparse_categorical_accuracy: 0.3386 - val_loss: 1.1037 - val_sparse_categorical_accuracy: 0.3408
Epoch 2/32
42/42 [==============================] - 16s 377ms/step - loss: 1.1013 - sparse_categorical_accuracy: 0.3455 - val_loss: 1.0984 - val_sparse_categorical_accuracy: 0.3515
Epoch 3/32
42/42 [==============================] - 16s 381ms/step - loss: 1.0933 - sparse_categorical_accuracy: 0.3688 - val_loss: 1.0872 - val_sparse_categorical_accuracy: 0.3738
Epoch 4/32
42/42 [==============================] - 16s 382ms/step - loss: 1.0721 - sparse_categorical_accuracy: 0.4199 - val_loss: 1.0423 - val_sparse_categorical_accuracy: 0.4810
Epoch 5/32
42/42 [==============================] - 16s 383ms/step - loss: 1.0214 - sparse_categorical_accuracy: 0.4995 - val_loss: 1.0235 - val_sparse_categorical_accuracy: 0.5066
Epoch 6/32
42/42 [==============================] - 18s 426ms/step - loss: 0.9958 - sparse_categorical_accura

2/2 [==============================] - 15s 7s/step

Inferencing
2/6 [=========>....................] - ETA: 0sWARNING:tensorflow:Callbacks method `on_predict_batch_end` is slow compared to the batch time (batch time: 0.0035s vs `on_predict_batch_end` time: 0.1775s). Check your callbacks.


6/6 [==============================] - 7s 1s/step

Fold 10 Accuracy: 0.6658


CV Mean Accuracy: 0.6716
CV StdDev Accuracy: 0.0342



In [45]:
df_test = pd.read_csv(config_2.PATH_TEST)

df_submission = pd.DataFrame({"id": df_test.id, "target": np.argmax(preds_test_2, axis = 1)})
df_submission.to_csv("submission3.csv", index = False)

df_submission.target.value_counts()

0    1877
1    1682
2    1636
Name: target, dtype: int64

In [46]:
df_submission.head()

,id,target
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,1
4,c32b0d16df,1


In [49]:
tost = pd.read_csv('test.csv.zip')
samp = pd.read_csv('sample_submission.csv')

In [50]:
samp.head()

,id,prediction
0,c6d58c3f69,1
1,cefcc82292,1
2,e98005252c,1
3,58518c10ba,1
4,c32b0d16df,1


    Things to attempt in the future (later today or tomorrow)
    Adding data augmentation
    Pre-processing text data before tokenization
    Trying other models
    Tuning hyperparameters
    Ensembling multiple models
